In [ ]:
# 개인 jupyter notebook 수행

In [ ]:
# 메서드 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import time

from sklearn.preprocessing import OneHotEncoder

In [ ]:
# 데이터 불러오기
user = pd.read_csv('C:/Users/cdbre/Desktop/MyLife/BigContest/data/user_spec.csv')
loan = pd.read_csv('C:/Users/cdbre/Desktop/MyLife/BigContest/data/loan_result.csv')

**user, loan 데이터 합치기**

In [ ]:
mul = pd.merge(loan[-loan.duplicated()], use, how='left',on='application_id')

In [ ]:
mul.to_csv('C:/Users/cdbre/Desktop/MyLife/BigContest/data/merge_user_loan.csv', index=False)

In [ ]:
# 데이터 불러오기
mul = pd.read_csv('C:/Users/cdbre/Desktop/MyLife/BigContest/data/merge_user_loan.csv')

In [ ]:
log = pd.read_csv('C:/Users/cdbre/Desktop/MyLife/BigContest/data/log_data.csv')

In [ ]:
org_mul = mul.copy()

In [ ]:
# loan에는 있지만 user에는 없는 유저 확인
mul[mul.user_id.isna()]

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,birth_year,gender,...,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt
68010,2015764,2022-04-29 15:16:36,27,193,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
615673,1669381,2022-03-01 18:03:44,14,197,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1694464,351899,2022-04-18 14:00:08,1,61,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2978293,1382876,2022-03-01 18:02:54,14,197,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3125378,2061284,2022-05-04 10:03:07,1,61,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13373160,197821,2022-03-02 10:29:03,13,262,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13386313,398321,2022-03-02 15:17:30,10,149,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13387271,1872250,2022-03-02 11:01:23,34,57,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13403427,2052437,2022-03-02 17:24:59,30,121,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
mul[mul.user_id.isna()].is_applied.value_counts()

1.0    113
Name: is_applied, dtype: int64

합해서 보면 loan에는 있는데 user에는 없는 유저가 113명 있음. 하지만 **모두 대출신청한 경우**이다. <br>
따라서 삭제하는게 좋을지 어떻게 해결해야될지 고민해야한다

In [ ]:
print('mul 테이블 모형 :', mul.shape)

mul 테이블 모형 : (13527337, 23)


In [ ]:
# 생성일시 date 형식으로 변환
mul["insert_time"] = pd.to_datetime(mul["insert_time"])
mul["loanapply_insert_time"] = pd.to_datetime(mul["loanapply_insert_time"])

**결측치 확인**

In [ ]:
df1 = pd.DataFrame(mul.isna().sum().sort_values(ascending=False), columns = ['count'])
df2 = pd.DataFrame((mul.isna().sum() / len(mul)).sort_values(ascending=False), columns = ['rate'])
df_mv = pd.concat([df1, df2], axis=1)
df_mv

,count,rate
personal_rehabilitation_complete_yn,11794071,0.871869
personal_rehabilitation_yn,5888807,0.435326
existing_loan_amt,3890275,0.287586
is_applied,3257223,0.240788
existing_loan_cnt,2685821,0.198548
credit_score,1509389,0.111581
company_enter_month,400450,0.029603
birth_year,128209,0.009478
gender,128209,0.009478
loan_limit,7495,0.000554


**train, test 분리 전 결측치 해결 가능**

**1) 생년월일(birth_year)과 성별(gender)의 결측치 128209개** 

In [ ]:
# 결측치가 12961개로 같은 gender, birth_year 변수 비교
set1 = set(mul[mul['gender'].isnull()].index)
set2 = set(mul[mul['birth_year'].isnull()].index)

print('생년월일과 성별의 결측치가 다른 것은 {}개'.format(len(set1 - set2)))

생년월일과 성별의 결측치가 다른 것은 0개


In [ ]:
# 생년월일 결측치인 데이터의 user_id의 빈도수 확인
pd.DataFrame(mul[mul['birth_year'].isna()].user_id.value_counts()).head()

,user_id
113797.0,609
786498.0,556
744785.0,380
374546.0,367
805180.0,339


In [ ]:
# 생년월일 결측치인 데이터의 user_id
df_no_birth = pd.DataFrame(mul[mul['birth_year'].isna()].user_id.value_counts())
print('생년월일 결측치 유저 아이디\n', df_no_birth.index)

# 생년월일 결측치가 아닌 데이터의 user_id
df_yes_birth = pd.DataFrame(mul[mul['birth_year'].notnull()].user_id.value_counts())
print('생년월일 결측치 아닌 유저 아이디\n', df_yes_birth.index)

생년월일 결측치 유저 아이디
 Float64Index([113797.0, 786498.0, 744785.0, 374546.0, 805180.0, 415037.0,
              560078.0, 301562.0, 518062.0, 850374.0,
              ...
              695806.0, 813320.0, 587513.0, 494113.0, 228922.0, 378968.0,
              170031.0, 594196.0, 491942.0,  14771.0],
             dtype='float64', length=5696)
생년월일 결측치 아닌 유저 아이디
 Float64Index([864179.0, 286813.0, 454280.0,  86932.0, 291823.0, 487389.0,
              451656.0, 549532.0,  62081.0, 482014.0,
              ...
              489105.0, 135892.0,  47962.0,  32109.0, 159794.0, 850982.0,
              130190.0, 172929.0, 868251.0, 390135.0],
             dtype='float64', length=312513)


In [ ]:
# 결측치이지만 결측치가 아닌 데이터에 존재하면 결측치를 메울 수 있다
here_list = [] ; not_here_list = [] ; here = 0 ; not_here = 0
for no_birth in df_no_birth.index:
    if no_birth in df_yes_birth.index:
        here += 1
        here_list.append(no_birth)
    else:
        not_here += 1
        not_here_list.append(no_birth)

print('결측값 채우기 {}개 가능'.format(here))
print('이 user는 채워진다(10개)', here_list[:10])
print('결측값 채우기 {}개 불가능'.format(not_here))
print('이 user는 안 채워진다(10개)', not_here_list[:10])

결측값 채우기 742개 가능
이 user는 채워진다(10개) [113797.0, 744785.0, 374546.0, 805180.0, 560078.0, 301562.0, 850374.0, 44773.0, 209257.0, 304996.0]
결측값 채우기 4954개 불가능
이 user는 안 채워진다(10개) [786498.0, 415037.0, 518062.0, 228468.0, 405016.0, 811727.0, 397812.0, 800189.0, 95949.0, 824694.0]


In [ ]:
# 처리 전 결측치
print('처리 전 결측치 \n', mul.isnull().sum()[['birth_year', 'gender']], '\n')

# birth_year와 gender 결측치 채우기
mul['birth_year'].fillna(mul.groupby('user_id')['birth_year'].transform('max'), inplace=True)
mul['gender'].fillna(mul.groupby('user_id')['gender'].transform('max'), inplace=True)

# 처리 후 결측치
print('처리 후 결측치 \n', mul.isnull().sum()[['birth_year', 'gender']])

처리 전 결측치 
 birth_year    128209
gender        128209
dtype: int64 

처리 후 결측치 
 birth_year    108340
gender        108340
dtype: int64


128209개의 결측치 **108340개로 감소**

In [ ]:
# 아예 생년월일과 성별이 누락된 유저들 예시 (20개)
print(not_here_list[:20])

[786498.0, 415037.0, 518062.0, 228468.0, 405016.0, 811727.0, 397812.0, 800189.0, 95949.0, 824694.0, 468619.0, 411420.0, 119292.0, 219103.0, 266722.0, 625243.0, 108328.0, 81799.0, 345517.0, 353794.0]


In [ ]:
# 일례로 유저 아이디가 786498.0인 경우 모든 생년월일과 성별 값이 결측치임을 확인할 수 있다
mul[mul.user_id == 786498.0]

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,birth_year,gender,...,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt
927022,1772463,2022-03-10 09:43:46,21,196,25000000.0,8.9,0.0,786498.0,NaN,NaN,...,EARNEDINCOME,201808.0,정규직,전월세,45000000.0,대환대출,NaN,NaN,NaN,NaN
927023,1772463,2022-03-10 09:43:45,44,8,39000000.0,16.3,0.0,786498.0,NaN,NaN,...,EARNEDINCOME,201808.0,정규직,전월세,45000000.0,대환대출,NaN,NaN,NaN,NaN
927024,1772463,2022-03-10 09:43:52,40,129,32000000.0,13.1,0.0,786498.0,NaN,NaN,...,EARNEDINCOME,201808.0,정규직,전월세,45000000.0,대환대출,NaN,NaN,NaN,NaN
927025,1772463,2022-03-10 09:43:45,14,197,16000000.0,10.9,0.0,786498.0,NaN,NaN,...,EARNEDINCOME,201808.0,정규직,전월세,45000000.0,대환대출,NaN,NaN,NaN,NaN
927026,1772463,2022-03-10 09:43:45,1,61,50000000.0,13.9,0.0,786498.0,NaN,NaN,...,EARNEDINCOME,201808.0,정규직,전월세,45000000.0,대환대출,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13205796,1777353,2022-03-08 01:18:44,21,196,25000000.0,8.9,0.0,786498.0,NaN,NaN,...,EARNEDINCOME,201808.0,정규직,전월세,55000000.0,생활비,NaN,NaN,NaN,NaN
13205797,1777353,2022-03-08 01:18:55,62,200,25000000.0,10.3,0.0,786498.0,NaN,NaN,...,EARNEDINCOME,201808.0,정규직,전월세,55000000.0,생활비,NaN,NaN,NaN,NaN
13205798,1777353,2022-03-08 01:18:53,27,176,45000000.0,10.9,0.0,786498.0,NaN,NaN,...,EARNEDINCOME,201808.0,정규직,전월세,55000000.0,생활비,NaN,NaN,NaN,NaN
13205799,1777353,2022-03-08 01:18:58,35,98,49000000.0,15.7,0.0,786498.0,NaN,NaN,...,EARNEDINCOME,201808.0,정규직,전월세,55000000.0,생활비,NaN,NaN,NaN,NaN


생년월일, 성별 안 쓴 사람들은 그냥 귀찮아서 안 쓴 것 같다. 이유를 잘 모르겠으니 한 번 생각해볼 필요가 있다 <br>
생년월일과 성별은 입력안했지만 대출은 받은 모습, 불균형 데이터라 대출 받은 경우를 삭제해버리면 **정보손실**임

**2) 대출 목적(purpose), 근로 형태(income_type), 고용 형태(employment_type), 주거소유 형태(houseown_type), 대출희망금액(desired_amount)의 결측치 113개**

In [ ]:
# 대출 목적의 경우 영어로 입력된 경우 있어 한글로 통합
mul.purpose.value_counts()

생활비             6828696
대환대출            4073346
주택구입             569679
전월세보증금           569284
사업자금             551570
기타               400084
투자               285670
LIVING           107532
자동차구입             58571
SWITCHLOAN        33615
HOUSEDEPOSIT      15912
BUYHOUSE          10989
BUSINESS           8513
ETC                7374
INVEST             3634
BUYCAR             2755
Name: purpose, dtype: int64

In [ ]:
mul.purpose = mul.purpose.replace('LIVING','생활비').replace('SWITCHLOAN','대환대출').replace('BUSINESS','사업자금').replace('ETC','기타').replace('HOUSEDEPOSIT','전월세보증금').replace('BUYHOUSE','주택구입').replace('INVEST','투자').replace('BUYCAR','자동차구입')

In [ ]:
# replace된 것 확인
mul.purpose.value_counts()

생활비       6936228
대환대출      4106961
전월세보증금     585196
주택구입       580668
사업자금       560083
기타         407458
투자         289304
자동차구입       61326
Name: purpose, dtype: int64

In [ ]:
# 고용형태의 경우
mul.employment_type.value_counts()

정규직    11065847
계약직     1191058
기타      1098232
일용직      172087
Name: employment_type, dtype: int64

In [ ]:
# 근로형태의 경우
mul.income_type.value_counts()

EARNEDINCOME       11467220
EARNEDINCOME2        564601
PRIVATEBUSINESS      494237
OTHERINCOME          402850
FREELANCER           312935
PRACTITIONER         285381
Name: income_type, dtype: int64

In [ ]:
# 주거소유형태의 경우
mul.houseown_type.value_counts()

전월세       6024265
자가        3817262
기타가족소유    3672895
배우자         12802
Name: houseown_type, dtype: int64

In [ ]:
# 전에는 "배우자" 선택하는 칸이 있었으나 현재는 없다. 기타가족소유에 통합
mul.houseown_type = mul.houseown_type.replace('배우자','기타가족소유')

In [ ]:
# 주거소유형태의 경우
mul.houseown_type.value_counts()

전월세       6024265
자가        3817262
기타가족소유    3685697
Name: houseown_type, dtype: int64

In [ ]:
# 대출 목적 결측치는 loan에만 있고 user에는 없는 경우
mul_null_113 = mul[mul.purpose.isna()]
mul_null_113

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,birth_year,gender,...,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt
68010,2015764,2022-04-29 15:16:36,27,193,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
615673,1669381,2022-03-01 18:03:44,14,197,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1694464,351899,2022-04-18 14:00:08,1,61,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2978293,1382876,2022-03-01 18:02:54,14,197,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3125378,2061284,2022-05-04 10:03:07,1,61,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13373160,197821,2022-03-02 10:29:03,13,262,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13386313,398321,2022-03-02 15:17:30,10,149,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13387271,1872250,2022-03-02 11:01:23,34,57,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13403427,2052437,2022-03-02 17:24:59,30,121,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
mul_null_113.is_applied.value_counts()

1.0    113
Name: is_applied, dtype: int64

**113개 결측치 경우 모두 대출 받음(loan에만 있는 신청서 번호)**

**3) 연소득(yearly_income) 결측치 119개**

In [ ]:
mul[mul.yearly_income.isna()]

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,birth_year,gender,...,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt
68010,2015764,2022-04-29 15:16:36,27,193,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
615673,1669381,2022-03-01 18:03:44,14,197,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1694464,351899,2022-04-18 14:00:08,1,61,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2978293,1382876,2022-03-01 18:02:54,14,197,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3125378,2061284,2022-05-04 10:03:07,1,61,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13373160,197821,2022-03-02 10:29:03,13,262,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13386313,398321,2022-03-02 15:17:30,10,149,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13387271,1872250,2022-03-02 11:01:23,34,57,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13403427,2052437,2022-03-02 17:24:59,30,121,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 연소득 결측치지만 유저 있는 경우
mul[(mul.yearly_income.isna()) & (mul.user_id.notnull())]

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,birth_year,gender,...,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt
4495385,341149,2022-06-30 21:32:20,64,254,45000000.0,10.3,NaN,670502.0,1981.0,1.0,...,OTHERINCOME,NaN,기타,기타가족소유,6000000.0,사업자금,0.0,0.0,1.0,3000000.0
4495387,341149,2022-06-30 21:32:19,13,262,35000000.0,16.4,NaN,670502.0,1981.0,1.0,...,OTHERINCOME,NaN,기타,기타가족소유,6000000.0,사업자금,0.0,0.0,1.0,3000000.0
4495388,341149,2022-06-30 21:32:19,38,113,5000000.0,19.5,NaN,670502.0,1981.0,1.0,...,OTHERINCOME,NaN,기타,기타가족소유,6000000.0,사업자금,0.0,0.0,1.0,3000000.0
4495389,341149,2022-06-30 21:32:19,14,128,15000000.0,12.5,NaN,670502.0,1981.0,1.0,...,OTHERINCOME,NaN,기타,기타가족소유,6000000.0,사업자금,0.0,0.0,1.0,3000000.0
4495392,341149,2022-06-30 21:32:19,29,265,10000000.0,9.1,NaN,670502.0,1981.0,1.0,...,OTHERINCOME,NaN,기타,기타가족소유,6000000.0,사업자금,0.0,0.0,1.0,3000000.0
4495394,341149,2022-06-30 21:32:19,57,71,23000000.0,15.4,NaN,670502.0,1981.0,1.0,...,OTHERINCOME,NaN,기타,기타가족소유,6000000.0,사업자금,0.0,0.0,1.0,3000000.0


In [ ]:
mul[mul.user_id == 670502.0][['yearly_income', 'insert_time', 'is_applied']]

,yearly_income,insert_time,is_applied
4495378,0.0,2022-06-30 22:41:11,NaN
4495379,0.0,2022-06-30 22:41:11,NaN
4495380,0.0,2022-06-30 22:41:11,NaN
4495381,0.0,2022-06-30 22:41:11,NaN
4495382,0.0,2022-06-30 22:41:11,NaN
4495383,0.0,2022-06-30 22:41:11,NaN
4495384,0.0,2022-06-30 22:41:11,NaN
4495385,NaN,2022-06-30 21:32:18,NaN
4495386,0.0,2022-06-30 22:41:11,NaN
4495387,NaN,2022-06-30 21:32:18,NaN


In [ ]:
# Groupby로 쓸 변수 추가
mul['date'] = mul['loanapply_insert_time'].apply(lambda x: format(datetime.strptime(x, '%Y-%m-%d %H:%M:%S'), '%Y-%m-%d'))

In [ ]:
# 670502.0 유저의 경우 연소득 계속 0 (계속 같은 날짜에 시도함) -> 따라서 0으로 대체함
mul['yearly_income'].fillna(mul.groupby(['user_id', 'date'])['yearly_income'].transform('mean'), inplace=True)

In [ ]:
# 결측치가 0으로 바뀜
mul[mul.user_id == 670502.0][['yearly_income', 'insert_time']]

,yearly_income,insert_time
4495378,0.0,2022-06-30 22:41:11
4495379,0.0,2022-06-30 22:41:11
4495380,0.0,2022-06-30 22:41:11
4495381,0.0,2022-06-30 22:41:11
4495382,0.0,2022-06-30 22:41:11
4495383,0.0,2022-06-30 22:41:11
4495384,0.0,2022-06-30 22:41:11
4495385,0.0,2022-06-30 21:32:18
4495386,0.0,2022-06-30 22:41:11
4495387,0.0,2022-06-30 21:32:18


In [ ]:
len(mul[mul.yearly_income.isna()])

113

119개의 결측치 **113개로 감소**

**여기까지 한거 csv로 저장**

In [ ]:
# csv로 저장
mul.to_csv('C:/Users/cdbre/Desktop/MyLife/BigContest/data/merge_user_loan_2.csv', index=False)

In [ ]:
mul = pd.read_csv('C:/Users/cdbre/Desktop/MyLife/BigContest/data/merge_user_loan_2.csv')

In [ ]:
mul.shape

(13527337, 24)

In [ ]:
mul.isna().sum()

application_id                                0
loanapply_insert_time                         0
bank_id                                       0
product_id                                    0
loan_limit                                 7495
loan_rate                                  7495
is_applied                              3257223
user_id                                     113
birth_year                               108340
gender                                   108340
insert_time                                 113
credit_score                            1509389
yearly_income                               113
income_type                                 113
company_enter_month                      400450
employment_type                             113
houseown_type                               113
desired_amount                              113
purpose                                     113
personal_rehabilitation_yn              5888807
personal_rehabilitation_complete_yn    1

**결측치 113개 제거 오피셜 !! <br>
또한 승인한도, 금리 결측치 뜨는 것도 제거 오피셜 !!**

In [ ]:
# loan에는 있지만 user에는 없어서 결측치 발생했던 113개 제거
mul.dropna(subset=['user_id'], axis=0, inplace=True)

In [ ]:
# 승인한도, 금리 결측치 제거
mul.dropna(subset=['loan_limit'], axis=0, inplace=True)

In [ ]:
# 결측치 확인
mul.isna().sum().sort_values(ascending=False)

personal_rehabilitation_complete_yn    11787469
personal_rehabilitation_yn              5885903
existing_loan_amt                       3888717
is_applied                              3255466
existing_loan_cnt                       2684722
credit_score                            1508618
company_enter_month                      399201
birth_year                               108176
gender                                   108176
application_id                                0
purpose                                       0
desired_amount                                0
houseown_type                                 0
employment_type                               0
yearly_income                                 0
income_type                                   0
loanapply_insert_time                         0
insert_time                                   0
user_id                                       0
loan_rate                                     0
loan_limit                              

In [ ]:
# 중간 전처리 확정 부분 저장
mul.to_csv('C:/Users/cdbre/Desktop/MyLife/BigContest/data/merge_user_loan_3.csv', index=False)